# Inter-Group Analysis - All Subjects and Saliency Data Predictions (Based on First Fixation Ground Truth Data, All Fixations Ground Truth Data, DeepGaze IIE - Saliency Model) 

Ground Truth Eye Tracking and DeepGaze IIE 

- Imports and path directions
- Sorting all ROIs according to their neuroanatomical order during visual saliency processing 
- Now mediate (arithmetic median) the results over the whole group. 
- One-Sample t-test for df_mean_roi_stats (DataFrame that stores over all ROIs, the corresponding Sal_model and the meaned r-values)
- One-Sample t-test for df_mean_roi_stats (DataFrame that stores for ROIs, the corresponding Sal_model and the meaned r-values)
- Visualize the r-values over all ROIs 
- Plot the mean correlation values for each ROI with significance stars and error bars
- Plot same barplot with confidence intervals and significance stars for ground truth eye tracking over all subjects.
- Plot same barplot with confidence intervals and significance stars for DeepGaze over all subjects. 
- Perform plots for single subjects 
- Rank df_mean_roi_stats for the highest and lowest ROI value 
- Paired t-tests: Is the correlation of the two saliency models (DeepGaze IIE or GroudTruthEyeTrackingData) significantly different?
- Repeated Measures ANOVA to evaluate the performance of all three models 
- Post-hoc: Paired t-tests (undirected), for each ROI, of the r values for DeepGaze and GroundTruthEyeTracking differ significantly across subjects.
- Model comparison averaged over all ROIs undirected paired t-tests 
- Now execution of directed t-tests
- Plotting of model difference (Mean r across all ROIs)

Written by Lisa Heinemann  
Last edited: 19/05/2025

Imports and Path directions

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
from scipy.stats import ttest_rel
from scipy.stats import t
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests


In [ ]:
# Set roots relative to the base_dir to gain access to the NSD Dataset, 
# Create a Path object for the base directory pub
base_dir = Path('/gpfs01/bartels/group/lheinemann/saliency-nsd-pub').expanduser()

# Second Base_dir for the PRF Model project 
base_dir_nsd = Path('/gpfs01/bartels/group/nsd_dataset')
############################################################################################################
NSD_DATA_ROOT  = base_dir_nsd / 'nsddata'

STIM_ROOT_NSD = NSD_DATA_ROOT / 'stimuli/nsd'
stim_file = STIM_ROOT_NSD / 'nsd_stimuli.hdf5'
shared1000_file = STIM_ROOT_NSD / 'shared1000.tsv'

exp_design_file = NSD_DATA_ROOT / 'experiments/nsd/nsd_expdesign.mat'
nsd_stiminfo_file = NSD_DATA_ROOT / 'experiments/nsd/nsd_stim_info_merged.csv'

print(nsd_stiminfo_file)
DATA_PATH = Path('/gpfs01/bartels/user/lheinemann/nsd-static-saliency/data')
print(DATA_PATH)

# Regions of Interest Path 
ROI_PATH = NSD_DATA_ROOT/ 'ppdata/'

#ROI_PATH = NSD_ROOT/ 'ppdata/'
DEVICE = 'cpu'

# Results Path for the Corrleation Plots
RESULTS_PATH = base_dir / f'results/Plots_Correlation_nsd_all_shared1000'
#RESULTS_PATH_subj = base_dir / f'results/Plots_Correlation_nsd_all_shared1000/subj0{sub_num}'


## Sorting all ROIs according to their neuroanatomical order during visual saliency processing 

See explanation of the sorting in Notebook 03_Analysis_corr_per_subject.ipynb

In [ ]:
# Count the number of rois 
all_rois = ['v1', 'v2', 'v3', 'v4', 'ofa', 'ffa1', 'ffa2', 'mtl_faces', 'atl', 'eba', 'fba1', 'fba2', 'mtl_bodies', 'opa', 'ppa',
          'rsc', 'owfa', 'vwfa_1', 'vwfa_2', 'mfs_words', 'mtl_words', 'vo1', 'vo2', 'phc1', 'phc2', 'mst', 
          'hmt', 'lo2', 'lo1', 'v3b', 'v3a', 'ips0', 'ips1', 'ips2', 'ips3', 'ips4', 'ips5', 'spl1', 'fef']

# Count the number of rois 
len(all_rois)

In [ ]:
# Align the ROIs to context relevant sorting - from early vision areas to higher order visual areas

# Define the order of ROIs based on the context
sorted_rois = [
    # Early visual areas - medial occipital
    'v1', 'v2', 'v3', 'v4',

    # Mid-level visual regions 
    'vo1', 'vo2', 'lo1', 'lo2',

    # Dorsal Regions
    'v3a', 'v3b',
    'hmt', 'mst',
    'ips0', 'ips1', 'ips2', 'ips3', 'ips4', 'ips5',

    # Higher-order processing areas 
    'spl1', 'phc1', 'phc2', 'fef', 
    #######################################################################
    # Higher-level visual areas (category selective: faces, bodies, words, places)
    # Faces
    'ofa', 'ffa1', 'ffa2', 'mtl_faces', 'atl', 
    # Bodies
    'eba', 'fba1', 'fba2', 'mtl_bodies',
    # Words
    'owfa', 'vwfa_1', 'vwfa_2', 'mfs_words', 'mtl_words',
    # Places
    'opa', 'ppa', 'rsc',
    ]

In [ ]:
# Define ROI categories and assign a color hue to each
roi_categories = {
    'Early visual areas':      (['v1', 'v2', 'v3', 'v4'], '#3465a4'), # dark blue
    'Mid-level visual regions':        (['vo1', 'vo2', 'lo1', 'lo2'], '#389bb7'), # light blue
    'Dorsal regions':          (['v3a', 'v3b', 'hmt', 'mst', 'ips0', 'ips1', 'ips2', 'ips3', 'ips4', 'ips5'], 'turquoise'),
    'Higher-order processing': (['spl1', 'phc1', 'phc2', 'fef'], '#66c2a5'), #greenish teal same as in other plot
    'Faces':                   (['ofa', 'ffa1', 'ffa2', 'mtl_faces', 'atl'], '#489775'), # soft dark green
    'Bodies':                  (['eba', 'fba1', 'fba2', 'mtl_bodies'], 'lightgreen'),
    'Words':                   (['owfa', 'vwfa_1', 'vwfa_2', 'mfs_words', 'mtl_words'], '#c1d27b'), #yellow-green
    'Places':                  (['opa', 'ppa', 'rsc'], '#fc8d62'), # orange same as in other plot
}

# Create a mapping from ROI to color
roi_color_map = {}
for category, (rois, color) in roi_categories.items():
    for roi in rois:
        roi_color_map[roi] = color


# Example: get color for each ROI in sorted_rois
roi_colors = [roi_color_map.get(roi) for roi in sorted_rois]

### Load the necessary data and indicate the paths 

In [ ]:
# Load paths to correlation results for ground truth eye-tracking data in every subject
gt_eyetracking_paths = [RESULTS_PATH / f'subj0{sub_num}/roi_means_subj0{sub_num}_GroundTruthEyeTracking.csv' \
                        for sub_num in range(1, 9)]

gt_eyetracking_paths_all_fix = [RESULTS_PATH / f'subj0{sub_num}/roi_means_subj0{sub_num}_AllFixations.csv' \
                        for sub_num in range(1, 9)]

# Load paths to correlation results for DeepGaze in every subject
deepgaze_data_paths = [RESULTS_PATH / f'subj0{sub_num}/roi_means_subj0{sub_num}_DeepGaze.csv' \
                    for sub_num in range(1, 9)]

# Now mediate (arithmetic median) the results over the whole group. 

In [ ]:
def aggregate_subject_roi_stats(deepgaze_data_paths, gt_eyetracking_paths, gt_eyetracking_paths_all_fix):
    """
    Parameters:
    - subject_files: List of file paths to subject-wise ROI stats CSVs.

    Returns:
    - df_mean_roi_stats: A DataFrame containing the mean r values and p-values across subjects for all ROIs.
    """

    # Read first and second column from CSV file and label them 'ROI' and 'pearsonr'. Prepend a column called
    # 'subject' and fill it with the subject number, also add a column called 'sal_model' and fill it with the saliency model name

    
    gt_df = pd.concat([pd.read_csv(path, usecols=[0, 1], names=['ROI', 'pearsonr'], header=None, skiprows=1).assign(subject=sub_num, sal_model='GroundTruthEyeTracking') \
                        for sub_num, path in enumerate(gt_eyetracking_paths, start=1)],  ignore_index=True)
    gt_df_all_fix = pd.concat([pd.read_csv(path, usecols=[0, 1], names=['ROI', 'pearsonr'], header=None, skiprows=1).assign(subject=sub_num, sal_model='AllFixations') \
                        for sub_num, path in enumerate(gt_eyetracking_paths_all_fix, start=1)],  ignore_index=True)
    dg_df = pd.concat([pd.read_csv(path, usecols=[0, 1], names=['ROI', 'pearsonr'], header=None, skiprows=1).assign(subject=sub_num, sal_model='DeepGaze') \
                    for sub_num, path in enumerate(deepgaze_data_paths, start=1)], ignore_index=True)
    ########################## Merge the two DataFrames on 'ROI' and 'subject' columns ################
    
    # Concatenate DeepGaze and GroundTruthEyeTracking DataFrames
    all_df = pd.concat([gt_df, gt_df_all_fix, dg_df], ignore_index=True)
    # Sort Rois by the order defined in sorted_rois
    all_df['ROI'] = pd.Categorical(all_df['ROI'], categories=sorted_rois, ordered=True)
    # Sort the DataFrame by the custom ROI order
    all_df = all_df.sort_values(by='ROI', ascending=True)
    # Sort by subject and sal_model for clear grouping
    all_df = all_df.sort_values(['subject', 'sal_model']).reset_index(drop=True)

    # Sort by ROI, subject, and add sal_model for clear grouping
    all_df = all_df.sort_values(['ROI', 'subject', 'sal_model']).reset_index(drop=True)

    # Add the standard deviation of the correlation coefficients for each ROI and saliency model
    all_df['std_pearsonr'] = all_df.groupby(['ROI', 'sal_model'])['pearsonr'].transform(lambda x: np.std(x, ddof=1))
    # Eddge case handling 
    all_df['std_pearsonr'] = all_df['std_pearsonr'].fillna(0)  # Fill NaN values with 0
    all_df['std_pearsonr'] = all_df['std_pearsonr'].replace([np.inf, -np.inf], 0)  # Replace inf values with 0


    ############################## Calculate DataFrame  ############################################################

    # Compute mean r and p-values across subjects for each ROI, and Saliency model - Using Fishers z-transformation
    df_mean_roi_stats = all_df.groupby(['ROI', 'sal_model']).agg(
        Mean_r=('pearsonr', lambda x: np.tanh(np.mean(np.arctanh(x.dropna()))))
    ).reset_index()

    # Ensure ROI column is categorical with the defined order
    df_mean_roi_stats['ROI'] = pd.Categorical(df_mean_roi_stats['ROI'], categories=sorted_rois, ordered=True)

    # Add the standard deviation of the correlation coefficients for each ROI and saliency model
    df_mean_roi_stats['std_pearsonr'] = all_df.groupby(['ROI', 'sal_model'])['std_pearsonr'].mean().values
    # Edges case handling
    df_mean_roi_stats['std_pearsonr'] = df_mean_roi_stats['std_pearsonr'].fillna(0)  # Fill NaN values with 0
    df_mean_roi_stats['std_pearsonr'] = df_mean_roi_stats['std_pearsonr'].replace([np.inf, -np.inf], 0)  # Replace inf values with 0

    # Sort the DataFrame by the custom ROI order
    df_mean_roi_stats = df_mean_roi_stats.sort_values(by='ROI', ascending=True)

    #Save the accumulated subjectwwise DataFrame to a CSV file
    all_df.to_csv(RESULTS_PATH / f'all_roi_all_subjects_all.csv', index=False)

    # Save the aggregated DataFrame to a CSV file
    df_mean_roi_stats.to_csv(RESULTS_PATH / f'all_roi_all_subjects_all_means.csv', index=False)
   
    return all_df, df_mean_roi_stats

In [ ]:
# Function call
all_df, df_mean_roi_stats = aggregate_subject_roi_stats(deepgaze_data_paths, gt_eyetracking_paths, gt_eyetracking_paths_all_fix)
print(df_mean_roi_stats.head())  # Display the first few rows of the aggregated DataFrame

In [ ]:
all_df

In [ ]:
df_mean_roi_stats

## One-Sample t-test for df_mean_roi_stats (DataFrame that stores over all ROIs, the corresponding Sal_model and the meaned r-values)

This tests is valid to test each model vs. 0. 
For each saliency model, is the average of its ROI-mean -correlations significantly greater than zero? 

In [ ]:
# One-sample t-test: Are mean r-values across ROIs significantly > 0 for each saliency model?
results = []
for sal_model in df_mean_roi_stats['sal_model'].unique():
    # Get mean r-values for all ROIs for this saliency model
    mean_r_vals = df_mean_roi_stats[df_mean_roi_stats['sal_model'] == sal_model]['Mean_r'].dropna()
    # Fisher z-transform for r-values
    z_vals = np.arctanh(mean_r_vals)
    # Perform one-sample t-test against 0 (randomness)
    t_stat, p_value = ttest_1samp(z_vals, popmean=0, alternative='greater')
    print(f"Saliency model: {sal_model} | t-stat: {t_stat:.3f} | p-value: {p_value:.4f}")
    results.append({'sal_model': sal_model, 't_stat': t_stat, 'p_value': p_value})

# Apply the FDR correction to the p-values
p_values = [res['p_value'] for res in results]

# Apply Holm Sidak correction to the p-values
_, corrected_p_values = multipletests(p_values, method='holm-sidak')[:2]
for i, res in enumerate(results):
    res['hs_p_value'] = corrected_p_values[i]
    res['significant'] = corrected_p_values[i] < 0.05
       
# Convert results to DataFrame for reporting or saving
t_test_results_df = pd.DataFrame(results)
print(t_test_results_df)

# Optionally, save to CSV
#t_test_results_df.to_csv(RESULTS_PATH / 'mean_r_across_rois_ttest_results.csv', index=False)

### Consistency with Paired T-Test Results

You mentioned that a paired t-test revealed that the 'all-fixations' condition outperformed the 'first fixations' condition. Let's consider how the one-sample t-test results (from cell 16) could align with this finding:

*   **Direct Comparison:** A paired t-test directly compares the two conditions (e.g., 'all-fixations' vs. 'first fixations' for the same subjects or items). This test is the most direct way to determine if one condition significantly outperforms the other.
*   **Individual Condition Performance:** The one-sample t-tests you performed assess each condition individually against a specific reference point (e.g., testing if the mean performance of 'all-fixations' is significantly different from zero, and separately, if the mean performance of 'first fixations' is significantly different from zero).

**How the one-sample t-tests can be consistent with the paired t-test:**

The results from the one-sample t-tests can be consistent with the paired t-test finding if, for example:

1.  The one-sample t-test for **'all-fixations'** shows a statistically significant positive mean (e.g., significantly greater than zero), indicating good performance.
2.  The one-sample t-test for **'first fixations'** shows:
    *   A positive mean that is also statistically significant but perhaps of a smaller magnitude than that for 'all-fixations', OR
    *   A mean that is not statistically significantly different from zero, suggesting its performance isn't reliably above the reference point, OR
    *   In some cases, even a mean that is significantly *less* than the reference if the outperformance is very stark.

If the 'all-fixations' condition demonstrates a notably stronger or more reliable positive effect in its individual one-sample t-test compared to the 'first fixations' condition, this would support and be consistent with the paired t-test conclusion that 'all-fixations' outperformed 'first fixations'.

While the one-sample t-tests look at each condition in isolation against a benchmark, their relative outcomes can provide corroborating evidence for the direct comparison made by the paired t-test.

## One-Sample t-test for df_mean_roi_stats (DataFrame that stores for ROIs, the corresponding Sal_model and the meaned r-values)

In [ ]:
# One-sample t-test for each single ROI (across subjects), for each saliency model
single_roi_ttest_results = []

for roi in df_mean_roi_stats['ROI'].unique():
    for sal_model in df_mean_roi_stats['sal_model'].unique():
        # Get mean r-values for this ROI and saliency model (across subjects)
        roi_rvals = all_df[
            (all_df['ROI'] == roi) &
            (all_df['sal_model'] == sal_model)
        ]['pearsonr'].dropna()
        if len(roi_rvals) > 1:  # t-test requires at least 2 values
            z_vals = np.arctanh(roi_rvals)
            t_stat, p_value = ttest_1samp(z_vals, popmean=0, alternative='greater')
            #print(f"ROI: {roi} | {sal_model}: t={t_stat:.3f}, p={p_value:.4f}, n={len(z_vals)}")
            single_roi_ttest_results.append({
                'ROI': roi,
                'sal_model': sal_model,
                't_stat': t_stat,
                'p_value': p_value,
                'n_subjects': len(z_vals)
            })
        else:
            print(f"ROI: {roi} | {sal_model}: Not enough subjects for t-test.")

# Apply the FDR correction to the p-values
p_values = [res['p_value'] for res in single_roi_ttest_results]
_, corrected_p_values = multipletests(p_values, method='fdr_bh')[:2]
for i, res in enumerate(single_roi_ttest_results):
    res['fdr_p_value'] = corrected_p_values[i]
    res['significant'] = corrected_p_values[i] < 0.05

# Convert to DataFrame for reporting or saving
df_roi_stats_fdr = pd.DataFrame(single_roi_ttest_results)

# Define the desired order for saliency models
model_order = ['GroundTruthEyeTracking', 'AllFixations', 'DeepGaze']

# Convert 'sal_model' to a categorical type with the specified order
# Ensure pandas (pd) is imported. If not, you might need to add 'import pandas as pd' earlier in the cell or notebook.
df_roi_stats_fdr['sal_model'] = pd.Categorical(df_roi_stats_fdr['sal_model'], categories=model_order, ordered=True)

# Sort the DataFrame first by ROI (to keep ROIs grouped) and then by the custom sal_model order
df_roi_stats_fdr = df_roi_stats_fdr.sort_values(by=['ROI', 'sal_model'])

print(df_roi_stats_fdr)

# Optionally, save to CSV
df_roi_stats_fdr.to_csv(RESULTS_PATH / 'df_roi_stats_fdr.csv', index=False)

In [ ]:
df_roi_stats_fdr
# Count the number of significant ROIs for each saliency model
significant_counts = df_roi_stats_fdr[df_roi_stats_fdr['significant']].groupby('sal_model')['ROI'].count()
print("Number of significant ROIs per saliency model:")
print(significant_counts)

# Print the p-values of the significant ROIs
significant_rois = df_roi_stats_fdr[df_roi_stats_fdr['significant']]
print("Significant ROIs and their p-values:")
print(significant_rois[['ROI', 'sal_model', 'fdr_p_value']])



In [ ]:
df_roi_stats_fdr


In [ ]:
all_df

## One-Sample *t*-test for df_mean_roi_stats (DataFrame that stores for ROIs, the corresponding Sal-model and the meaned r-values)

In [ ]:
""" # Define ROI categories (already in your code)
roi_categories = {
    "Visual ROIs": ["v1", "v2", "v3", "v4"],
    "FLOC Faces ROIs": ["ofa", "ffa1", "ffa2", "atl"],
    "FLOC Words ROIs": ["eba", "fba1", "fba2"],
    "FLOC Places ROIs": ["opa", "ppa", "rsc"],
    "FLOC Bodies ROIs": ["owfa", "vwfa_1", "vwfa_2", "mfs_words", "mtl_words"],
    "Ventral Temporal ROIs": ["vo1", "vo2", "phc1", "phc2"],
    "Dorso Lateral ROIs": ["v3a", "v3b", "lo2", "lo1"],
    "Parietal Frontal ROIs": ["ips0", "ips1", "ips2", "ips3", "ips4", "ips5", "spl1", "fef"]
}

group_ttest_results = []

for group_name, roi_list in roi_categories.items():
    for sal_model in df_mean_roi_stats['sal_model'].unique():
        # Select mean r-values for this group and saliency model
        group_rvals = df_mean_roi_stats[
            (df_mean_roi_stats['ROI'].isin(roi_list)) &
            (df_mean_roi_stats['sal_model'] == sal_model)
        ]['Mean_r'].dropna()
        if len(group_rvals) > 1:  # t-test requires at least 2 values
            z_vals = np.arctanh(group_rvals)
            t_stat, p_value = ttest_1samp(z_vals, popmean=0, alternative='greater')
            print(f"{group_name} | {sal_model}: t={t_stat:.3f}, p={p_value:.4f}, n={len(z_vals)}")
            group_ttest_results.append({
                'Group': group_name,
                'sal_model': sal_model,
                't_stat': t_stat,
                'p_value': p_value,
                'n_rois': len(z_vals)
            })
        else:
            print(f"{group_name} | {sal_model}: Not enough ROIs for t-test.")

# Convert to DataFrame for reporting or saving
group_ttest_results_df = pd.DataFrame(group_ttest_results)
print(group_ttest_results_df)

# Optionally, save to CSV
#group_ttest_results_df.to_csv(RESULTS_PATH / 'group_mean_r_ttest_results.csv', index=False) """

## Visualize the r-values over all ROIs 

Matplotlib 

In [ ]:
# Plot the mean correlation values for each ROI with significance stars
def plot_mean_correlation_with_significance_all_subjects(df_mean_roi_stats,df_roi_stats_fdr):
    """
    Plot the mean correlation values for each ROI with significance stars.

    Parameters:
    - df_mean_roi_stats: DataFrame containing mean r values across subjects for all ROIs.
    - df_roi_stats_fdr: DataFrame containing FDR-corrected p-values for each ROI.
    """
    # Extract ROI names, mean r values, and mean p-values
    rois = df_mean_roi_stats['ROI']
    mean_r_values = df_mean_roi_stats['Mean_r']
    p_values = df_roi_stats_fdr['fdr_p_value']

    # Create the bar plot
    plt.figure(figsize=(20, 10)) # Adjust the width and height of the overall figure size 
    bars = plt.bar(rois, mean_r_values, color='skyblue', edgecolor='black')

    # Adjust the y-limit to create space for significance stars
    min_r_value = min(mean_r_values)
    max_r_value = max(mean_r_values)
    plt.ylim(min_r_value - 0.05, max_r_value + 0.02)  # Add extra space above and below the bars

    # Add significance stars
    for i, (bar, p_value) in enumerate(zip(bars, p_values)):
        if p_value < 0.001:
            significance = '***'
        elif p_value < 0.01:
            significance = '**'
        elif p_value < 0.05:
            significance = '*'
        else:
            significance = ''
        
        # Add the stars above the bar
        if significance:
            plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                     significance, ha='center', va='bottom', fontsize=12, color='red')

    # Add labels and title
    plt.xlabel('ROIs', fontsize=7)
    plt.ylabel('Mean Correlation (r)', fontsize=10)
    plt.title(f'Mean Correlation with Significance Across Subjects - Sal_mod_{sal_model}', fontsize=14)

In [ ]:
# Function call
plot_mean_correlation_with_significance_all_subjects(df_mean_roi_stats,df_roi_stats_fdr)
plt.xticks(rotation=60, fontsize=9)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the plot to a file
plt.savefig(RESULTS_PATH / f'mean_correlation_with_significance_all_subjects_fdr_corrected_{sal_model}.png', dpi=300)

# Show the plot
plt.show()

# Plot the mean correlation values for each ROI with significance stars and error bars 

In [ ]:
# Add the fdr corrected p values of df_roi_stats_fdr to all_df dataframe according to the roi 
# and sal_model to create a comprhensive plot 
############################################################################

# Create a lookup for p-values
pval_lookup = {(row['ROI'], row['sal_model']): row['fdr_p_value'] for _, row in df_roi_stats_fdr.iterrows()}
# Add the p-values to all_df
all_df['fdr_p_value'] = all_df.apply(lambda row: pval_lookup.get((row['ROI'], row['sal_model'])), axis=1)

print(all_df.head())  # Display the first few rows of the updated DataFrame
print(all_df.tail()) # Display the last few rows of the updated DataFrame

# Save the updated DataFrame with FDR correction results
# all_df.to_csv(RESULTS_PATH / f'roi_means_all_subjects_fdr_corrected.csv', index=False)

In [ ]:
""" # With automatic error bar, the errorbar printed in the plot below is more accurate

sns.catplot(data=all_df, x='ROI', y='pearsonr', hue='sal_model',
            kind='bar', height=6, aspect=2, palette='Set2', 
            estimator=lambda x: np.tanh(np.mean(np.arctanh(x))))
# save the plot to a file
#plt.savefig(RESULTS_PATH / f'combined_mean_correlation_sig_all_subjects_fdr_corrected.png', dpi=300)  """

In [ ]:
all_df

## Plot same barplot with confidence intervals and significance stars for ground truth eye tracking over all subjects. 

Blue-Green-Purple-Lightred-Orange-Gold Design for Plots 

In [ ]:
""" roi_categories = {
    'Early visual areas':      (['v1', 'v2', 'v3', 'v4'], '#3465a4'), # navy blue
    'Mid-level visual':        (['vo1', 'vo2', 'lo1', 'lo2'], '#389bb7'), # light blue
    'Dorsal regions':          (['v3a', 'v3b', 'hmt', 'mst', 'ips0', 'ips1', 'ips2', 'ips3', 'ips4', 'ips5'], '#66c2a5'), #greenish teal same as in other plot
    'Higher-order processing': (['spl1', 'phc1', 'phc2', 'fef'], '#b2df8a'), # green
    'Faces':                   (['ofa', 'ffa1', 'ffa2', 'mtl_faces', 'atl'],  '#b07aa1'), #purple 
    'Bodies':                  (['eba', 'fba1', 'fba2', 'mtl_bodies'], '#ff9999'), # light red
    'Words':                   (['owfa', 'vwfa_1', 'vwfa_2', 'mfs_words', 'mtl_words'], '#fc8d62'), # orange same as in other plot 
    'Places':                  (['opa', 'ppa', 'rsc'], '#f7c873'), # gold
}
# Create a mapping from ROI to color
roi_color_map = {}
for category, (rois, color) in roi_categories.items():
    for roi in rois:
        roi_color_map[roi] = color


# Example: get color for each ROI in sorted_rois
roi_colors = [roi_color_map.get(roi) for roi in sorted_rois] """

Green-Blue-Turqoise-Orange Design for Plots 

In [ ]:
roi_categories = {
    'Early visual areas':      (['v1', 'v2', 'v3', 'v4'], '#3465a4'), # dark blue
    'Mid-level visual regions':        (['vo1', 'vo2', 'lo1', 'lo2'], '#56B4E9'), # light blue
    'Dorsal regions':          (['v3a', 'v3b', 'hmt', 'mst', 'ips0', 'ips1', 'ips2', 'ips3', 'ips4', 'ips5'], 'turquoise'),
    'Higher-order processing': (['spl1', 'phc1', 'phc2', 'fef'], '#f3e5ab'), #greenish teal same as in other plot
    'Faces':                   (['ofa', 'ffa1', 'ffa2', 'mtl_faces', 'atl'], '#489775'), # soft dark green
    'Bodies':                  (['eba', 'fba1', 'fba2', 'mtl_bodies'], 'lightgreen'),
    'Words':                   (['owfa', 'vwfa_1', 'vwfa_2', 'mfs_words', 'mtl_words'], '#c1d27b'), #yellow-green
    'Places':                  (['opa', 'ppa', 'rsc'], '#fc8d62'), # orange same as in other plot
}

# Create a mapping from ROI to color
roi_color_map = {}
for category, (rois, color) in roi_categories.items():
    for roi in rois:
        roi_color_map[roi] = color


# Example: get color for each ROI in sorted_rois
roi_colors = [roi_color_map.get(roi) for roi in sorted_rois]

In [ ]:
all_df

In [ ]:
eytracking_df = all_df[all_df['sal_model'] == 'GroundTruthEyeTracking'].copy()

#sns.set_theme(style="whitegrid")
# Map ROI to category
roi_to_category = {}
for category, (rois, _) in roi_categories.items():
    for roi in rois:
        roi_to_category[roi] = category
eytracking_df['roi_category'] = eytracking_df['ROI'].map(roi_to_category)

# Use hue and a palette mapping category to color
category_palette = {cat: color for cat, (_, color) in roi_categories.items()}

plt.figure(figsize=(20, 8))
ax = sns.barplot(
    data=eytracking_df, x='ROI', y='pearsonr',
    hue='roi_category', palette=category_palette,
    estimator=lambda x: np.tanh(np.mean(np.arctanh(x)))
)
# Ensure gridlines are behind the bars
ax.set_axisbelow(True)
# Add vertical gridlines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.xlabel("ROI")
plt.ylabel("Mean Correlation (r)")
plt.title("Mean Correlation for All Subjects per ROI - Ground Truth Eye Tracking First Fixations)")


#######################################################################################################################################
# Define custom ROI label mapping - so that the labels of the plot are coherent with the thesis and literature 
roi_label_map = {
    'v4': 'hV4',
    'hmt': 'hMT',
    'mtl_faces': 'MTL-faces',
    'mtl_bodies': 'MTL-bodies',
    'mfs_words': 'MFS-words',
    'mtl_words': 'MTL-words',
    'owfa': 'OVWFA',
    # Add more mappings if needed
}

# Get current x-tick labels (original ROI names)
original_roi_order = [label.get_text() for label in ax.get_xticklabels()]

# Apply custom label mapping for display
custom_labels = [roi_label_map.get(lbl, lbl.upper()) for lbl in original_roi_order]
ax.set_xticklabels(custom_labels, rotation=60)

plt.tight_layout()
plt.ylim(-0.1,0.18)

# Move the legend to the top right corner
if ax.legend_ is not None:
    ax.legend_.set_title("ROI category")
    ax.legend_.set_bbox_to_anchor((1.05, 1))

""" # Use original ROI names for lookup, but display custom labels
roi_order = original_roi_order
saliency_order = [t.get_text() for t in g._legend.texts]
n_roi = len(roi_order)
n_sal = len(saliency_order) """
#######################################################################################################################################
# Add significance stars using FDR-corrected p-values
for i, bar in enumerate(ax.patches[:len(original_roi_order)]):
    roi = original_roi_order[i]
    match = eytracking_df[eytracking_df['ROI'] == roi]
    if match.empty or pd.isnull(match['fdr_p_value'].values[0]):
        continue
    p_value = match['fdr_p_value'].values[0]
    if p_value < 0.001:
        significance = '***'
    elif p_value < 0.01:
        significance = '**'
    elif p_value < 0.05:
        significance = '*'
    else:
        significance = ''
    if significance:
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.04,  # Print higher above the bar
            significance,
            ha='center', va='bottom',
            fontsize=14, color='red'  # Changed color and font size
        )

# Save the plot to a file
plt.savefig(RESULTS_PATH / 'ground_truth_eye_tracking_mean_correlation_sig_all_subjects_fdr_corrected.png', dpi=300, bbox_inches='tight')
plt.savefig(RESULTS_PATH / 'ground_truth_eye_tracking_mean_correlation_sig_all_subjects_fdr_corrected.svg', bbox_inches='tight')

plt.show()

## Plot same barplot with confidence intervals and significance stars for DeepGaze over all subjects. 

In [ ]:
deepgaze_df = all_df[all_df['sal_model'] == 'DeepGaze'].copy()

# Map ROI to category
roi_to_category = {}
for category, (rois, _) in roi_categories.items():
    for roi in rois:
        roi_to_category[roi] = category
deepgaze_df['roi_category'] = deepgaze_df['ROI'].map(roi_to_category)

# Use hue and a palette mapping category to color
category_palette = {cat: color for cat, (_, color) in roi_categories.items()}

plt.figure(figsize=(20, 8))
ax = sns.barplot(
    data=deepgaze_df, x='ROI', y='pearsonr',
    hue='roi_category', palette=category_palette,
    estimator=lambda x: np.tanh(np.mean(np.arctanh(x)))
)
# Ensure gridlines are behind the bars
ax.set_axisbelow(True)

# Add vertical gridlines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.xlabel("ROI")
plt.ylabel("Mean Correlation (r)")
plt.title("Mean Correlation for All Subjects per ROI - DeepGaze IIE")

#######################################################################################################################################
# Define custom ROI label mapping - so that the labels of the plot are coherent with the thesis and literature 
roi_label_map = {
    'v4': 'hV4',
    'hmt': 'hMT',
    'mtl_faces': 'MTL-faces',
    'mtl_bodies': 'MTL-bodies',
    'mfs_words': 'MFS-words',
    'mtl_words': 'MTL-words',
    'owfa': 'OVWFA',
    # Add more mappings if needed
}

# Get current x-tick labels (original ROI names)
original_roi_order = [label.get_text() for label in ax.get_xticklabels()]

# Apply custom label mapping for display
custom_labels = [roi_label_map.get(lbl, lbl.upper()) for lbl in original_roi_order]
ax.set_xticklabels(custom_labels, rotation=60)

plt.tight_layout()
plt.ylim(-0.1,0.18)

# Move the legend to the top right corner
if ax.legend_ is not None:
    ax.legend_.set_title("ROI category")
    ax.legend_.set_bbox_to_anchor((1.05, 1))

""" # Use original ROI names for lookup, but display custom labels
roi_order = original_roi_order
saliency_order = [t.get_text() for t in g._legend.texts]
n_roi = len(roi_order)
n_sal = len(saliency_order) """
#######################################################################################################################################
# Add significance stars using FDR-corrected p-values
for i, bar in enumerate(ax.patches[:len(original_roi_order)]):
    roi = original_roi_order[i]
    match = deepgaze_df[deepgaze_df['ROI'] == roi]
    if match.empty or pd.isnull(match['fdr_p_value'].values[0]):
        continue
    p_value = match['fdr_p_value'].values[0]
    if p_value < 0.001:
        significance = '***'
    elif p_value < 0.01:
        significance = '**'
    elif p_value < 0.05:
        significance = '*'
    else:
        significance = ''
    if significance:
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.045,  # Print higher above the bar
            significance,
            ha='center', va='bottom',
            fontsize=14, color='red'  # Changed color and font size
        )

# Save the plot to a file
plt.savefig(RESULTS_PATH / 'deepgaze_mean_correlation_sig_all_subjects_fdr_corrected.png', dpi=300, bbox_inches='tight')
plt.savefig(RESULTS_PATH / 'deepgaze_mean_correlation_sig_all_subjects_fdr_corrected.svg', bbox_inches='tight')
plt.show()

same overall plot for all fixations 

In [ ]:
deepgaze_df = all_df[all_df['sal_model'] == 'AllFixations'].copy()

# Map ROI to category
roi_to_category = {}
for category, (rois, _) in roi_categories.items():
    for roi in rois:
        roi_to_category[roi] = category
deepgaze_df['roi_category'] = deepgaze_df['ROI'].map(roi_to_category)

# Use hue and a palette mapping category to color
category_palette = {cat: color for cat, (_, color) in roi_categories.items()}

plt.figure(figsize=(20, 8))
ax = sns.barplot(
    data=deepgaze_df, x='ROI', y='pearsonr',
    hue='roi_category', palette=category_palette,
    estimator=lambda x: np.tanh(np.mean(np.arctanh(x)))
)

# Ensure gridlines are behind the bars
ax.set_axisbelow(True)
# Add vertical gridlines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.xlabel("ROI")
plt.ylabel("Mean Correlation (r)")
plt.title("Mean Correlation for All Subjects per ROI - All Fixations")

#######################################################################################################################################
# Define custom ROI label mapping - so that the labels of the plot are coherent with the thesis and literature 
roi_label_map = {
    'v4': 'hV4',
    'hmt': 'hMT',
    'mtl_faces': 'MTL-faces',
    'mtl_bodies': 'MTL-bodies',
    'mfs_words': 'MFS-words',
    'mtl_words': 'MTL-words',
    'owfa': 'OVWFA',
    # Add more mappings if needed
}

# Get current x-tick labels (original ROI names)
original_roi_order = [label.get_text() for label in ax.get_xticklabels()]

# Apply custom label mapping for display
custom_labels = [roi_label_map.get(lbl, lbl.upper()) for lbl in original_roi_order]
ax.set_xticklabels(custom_labels, rotation=60)

plt.tight_layout()
plt.ylim(-0.1,0.18)

# Move the legend to the top right corner
if ax.legend_ is not None:
    ax.legend_.set_title("ROI category")
    ax.legend_.set_bbox_to_anchor((1.05, 1))

""" # Use original ROI names for lookup, but display custom labels
roi_order = original_roi_order
saliency_order = [t.get_text() for t in g._legend.texts]
n_roi = len(roi_order)
n_sal = len(saliency_order) """

#######################################################################################################################################
# Add significance stars using FDR-corrected p-values
for i, bar in enumerate(ax.patches[:len(original_roi_order)]):
    roi = original_roi_order[i]
    match = deepgaze_df[deepgaze_df['ROI'] == roi]
    if match.empty or pd.isnull(match['fdr_p_value'].values[0]):
        continue
    p_value = match['fdr_p_value'].values[0]
    if p_value < 0.001:
        significance = '***'
    elif p_value < 0.01:
        significance = '**'
    elif p_value < 0.05:
        significance = '*'
    else:
        significance = ''
    if significance:
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.045,  # Print higher above the bar
            significance,
            ha='center', va='bottom',
            fontsize=14, color='red'  # Changed color and font size
        )

# Save the plot to a file
plt.savefig(RESULTS_PATH /'allFix_mean_correlation_sig_all_subjects_fdr_corrected.png', dpi=300, bbox_inches='tight')
plt.savefig(RESULTS_PATH /'allFix_mean_correlation_sig_all_subjects_fdr_corrected.svg', bbox_inches='tight')
plt.show()

# Perform plots for single subjects 

In [ ]:
# Define the order of ROIs based on the context
sorted_rois = [
    # Early visual areas - medial occipital
    'v1', 'v2', 'v3', 'v4',

    # Mid-level visual regions 
    'vo1', 'vo2', 'lo1', 'lo2',

    # Dorsal regions
    'v3a', 'v3b',
    'hmt', 'mst',
    'ips0', 'ips1', 'ips2', 'ips3', 'ips4', 'ips5',

    # Higher-order processing areas 
    'spl1', 'phc1', 'phc2', 'fef', 
    #######################################################################
    # Higher-level visual areas (category selective: faces, bodies, words, places)

    # Faces
    'ofa', 'ffa1', 'ffa2', 'mtl_faces', 'atl', 
    # Bodies
    'eba', 'fba1', 'fba2', 'mtl_bodies',
    # Words
    'owfa', 'vwfa_1', 'vwfa_2', 'mfs_words', 'mtl_words',
    # Places
    'opa', 'ppa', 'rsc',
    ]

In [ ]:
for sub_num in range(1, 9):

    # Subjectwise Data Path Storage

    DATA_PATH_subj = DATA_PATH / f'subjectwise_data/subject0{sub_num}'
    
    # Load the numpy array that store voxelwise correlation values on the main diagonal

    corr_md_subj_all_rois = np.load(DATA_PATH_subj/ f'corr_mat_all_rois_subj0{sub_num}.npy', allow_pickle = True) 

    corr_md_subj_all_rois_deepgaze = np.load(DATA_PATH_subj/ f'corr_mat_all_rois_deepgazeIIE_subj0{sub_num}.npy', allow_pickle = True)

    corr_md_subj_all_rois_allFix = np.load(DATA_PATH_subj/ f'corr_mat_all_rois_allfixations_subj0{sub_num}.npy', allow_pickle = True)
    # Convert corr_md_subj_all_rois (assumed 2D numpy array) to a DataFrame
    corr_md_subj_all_rois_df = pd.DataFrame(corr_md_subj_all_rois, columns=["ROI", "r_value","p_value"])
    # Inspect the shape and create the DataFrame with the correct number of columns
    print("corr_md_subj_all_rois shape:", corr_md_subj_all_rois.shape)

    # If shape is (N, 3), add a third column name (e.g., "p_value")
    corr_md_subj_all_rois_df = pd.DataFrame(
        corr_md_subj_all_rois, columns=["ROI", "r_value", "p_value"]

    )
    print(corr_md_subj_all_rois_df.head())

    # Save the subjectwise DataFrame to a CSV file
    corr_md_subj_all_rois_df.to_csv(DATA_PATH_subj / f'corr_mat_all_rois_ET_first_subj0{sub_num}.csv', index=False)
    ############################################################ DeepGaze ###################################################
    # Convert corr_md_subj_all_rois (assumed 2D numpy array) to a DataFrame
    corr_md_subj_all_rois_df_dg = pd.DataFrame(corr_md_subj_all_rois_deepgaze, columns=["ROI", "r_value","p_value"])
    # Inspect the shape and create the DataFrame with the correct number of columns
    print("corr_md_subj_all_rois shape:", corr_md_subj_all_rois_deepgaze.shape)

    # If shape is (N, 3), add a third column name (e.g., "p_value")
    corr_md_subj_all_rois_df_dg = pd.DataFrame(
        corr_md_subj_all_rois_deepgaze, columns=["ROI", "r_value", "p_value"]
    )

    print(corr_md_subj_all_rois_df_dg.head())

    # Save the subjectwise DataFrame to a CSV file
    corr_md_subj_all_rois_df_dg.to_csv(DATA_PATH_subj / f'corr_mat_all_rois_deepgazeIIE_subj0{sub_num}.csv', index=False)

    ########################################################### All Fixations ###################################################
    # Convert corr_md_subj_all_rois (assumed 2D numpy array) to a DataFrame
    corr_md_subj_all_rois_df_allFix = pd.DataFrame(corr_md_subj_all_rois_allFix, columns=["ROI", "r_value","p_value"])
    # Inspect the shape and create the DataFrame with the correct number of columns
    print("corr_md_subj_all_rois shape:", corr_md_subj_all_rois_allFix.shape)

    # If shape is (N, 3), add a third column name (e.g., "p_value")
    corr_md_subj_all_rois_df_allFix = pd.DataFrame(
        corr_md_subj_all_rois_allFix, columns=["ROI", "r_value", "p_value"]
    )

    print(corr_md_subj_all_rois_df_allFix.head())

    # Save the subjectwise DataFrame to a CSV file
    corr_md_subj_all_rois_df_allFix.to_csv(DATA_PATH_subj / f'corr_mat_all_rois_allFix_subj0{sub_num}.csv', index=False)

In [ ]:
corr_md_subj_all_rois_df_dg
corr_md_subj_all_rois_df
corr_md_subj_all_rois_df_allFix

In [ ]:
for sub_num in range(1, 9):
    # Subjectwise Data Path Storage
    DATA_PATH_subj = DATA_PATH / f'subjectwise_data/subject0{sub_num}'
    # Filter for DeepGaze and the current subject
    eytracking_df = pd.read_csv(DATA_PATH_subj / f'corr_mat_all_rois_ET_first_subj0{sub_num}.csv')
    if eytracking_df.empty:    

        print(f"No data for subject {sub_num}")
        continue
   
    # Map ROI to category
    roi_to_category = {}
    for category, (rois, _) in roi_categories.items():
        for roi in rois:
            roi_to_category[roi] = category
    eytracking_df['roi_category'] = eytracking_df['ROI'].map(roi_to_category)

    # Create a grouped ROI order: all ROIs grouped by category order
    grouped_roi_order = []
    for category, (rois, _) in roi_categories.items():
        grouped_roi_order.extend(rois)
    # Filter to only those present in the DataFrame
    grouped_roi_order = [roi for roi in grouped_roi_order if roi in eytracking_df['ROI'].values]


    # Use hue and a palette mapping category to color
    category_palette = {cat: color for cat, (_, color) in roi_categories.items()}

    plt.figure(figsize=(20, 8))
    ax = sns.barplot(
        data=eytracking_df, x='ROI', y='r_value',
        hue='roi_category', palette=category_palette,
        estimator=lambda x: np.tanh(np.mean(np.arctanh(x))),
        errorbar='sd',  # Use standard deviation for error bars
        order=grouped_roi_order
    )
    # Ensure gridlines are behind the bars
    ax.set_axisbelow(True)

    # Add vertical gridlines
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    plt.xlabel("ROI")
    plt.ylabel("Mean Correlation (r)")
    plt.title(f"Mean Correlation per ROI - Eye Tracking First Fixation, Subject 0{sub_num}")


    #######################################################################################################################################
    # Define custom ROI label mapping - so that the labels of the plot are coherent with the thesis and literature 
    roi_label_map = {
        'v4': 'hV4',
        'hmt': 'hMT',
        'mtl_faces': 'MTL-faces',
        'mtl_bodies': 'MTL-bodies',
        'mfs_words': 'MFS-words',
        'mtl_words': 'MTL-words',
        'owfa': 'OVWFA',
        # Add more mappings if needed
    }

    # Get current x-tick labels (original ROI names)
    original_roi_order = [label.get_text() for label in ax.get_xticklabels()]

    # Apply custom label mapping for display
    custom_labels = [roi_label_map.get(lbl, lbl.upper()) for lbl in original_roi_order]
    ax.set_xticklabels(custom_labels, rotation=60)

    plt.tight_layout()
    plt.ylim(-0.1,0.37)

    # Move the legend to the top right corner
    if ax.legend_ is not None:
        ax.legend_.set_title("ROI category")
        ax.legend_.set_bbox_to_anchor((1.05, 1))

    plt.xticks(rotation=60)
    plt.ylim(-0.1, 0.37)
    plt.tight_layout()

    # Get the order of ROIs and hues as plotted
    roi_order = [label.get_text() for label in ax.get_xticklabels()]
    hue_order = list(deepgaze_df['roi_category'].dropna().unique())
    n_hue = len(hue_order)
    n_roi = len(roi_order)

    ######################################################################################################################################
    # Load the fdr_corrected p_values per ROI and subject from the subjectwiese df geneated in 03_analysis, while performing the ttest
    # Create a lookup for p-values

    for sub_num_inner_loop in range(1, 9): # Renamed to avoid conflict with outer loop sub_num
        # Define the subject-specific results path dynamically 
        RESULTS_PATH_subj = RESULTS_PATH / f'subj0{sub_num_inner_loop}'
        # Load the DataFrame with FDR-corrected p-values
        # Assuming sal_model is defined elsewhere or you want to use a specific one
        # For this example, I'll use a placeholder. You might need to adjust this.
        current_sal_model = 'GroundTruthEyeTracking' # Placeholder
        df_singlesub_fdr= pd.read_csv(RESULTS_PATH_subj / f'roi_means_subj0{sub_num_inner_loop}_{current_sal_model}_fdr_corrected.csv', index_col=False)

    # Save the updated DataFrame with FDR correction results
    
    # Add significance stars using p-values
    for i, bar in enumerate(ax.patches[:len(original_roi_order)]):
        roi = original_roi_order[i]
        match = df_singlesub_fdr[df_singlesub_fdr['ROI'] == roi]
        
    
        if match.empty or pd.isnull(match['FDR-corrected p-value'].values[0]):
            continue
        p_value = match['FDR-corrected p-value'].values[0]
        # Only print stars if the correlation (bar height) is positive
        if bar.get_height() > 0:
            if p_value < 0.001:
                significance = '***'
            elif p_value < 0.01:
                significance = '**'
            elif p_value < 0.05:
                significance = '*'
            else:
                significance = ''
            if significance:
                ax.text(
                    bar.get_x() + bar.get_width() / 2,
                    bar.get_height() + 0.01, # Adjusted y-offset for better placement
                    significance,
                    ha='center', va='bottom',
                    fontsize=14, color='red'
                )

    plt.savefig(RESULTS_PATH / f'eyetracking_firstFixations_mean_correlation_sig_subj0{sub_num}.png', dpi=300, bbox_inches='tight')
    plt.savefig(RESULTS_PATH / f'eyetracking_firstFixations_mean_correlation_sig_subj0{sub_num}.svg', bbox_inches='tight')
    plt.show()

For Groundtruth all fixations - plots per subj 

In [ ]:
for sub_num in range(1, 9):
    # Subjectwise Data Path Storage
    DATA_PATH_subj = DATA_PATH / f'subjectwise_data/subject0{sub_num}'
    # Filter for DeepGaze and the current subject
    eytracking_df = pd.read_csv(DATA_PATH_subj / f'corr_mat_all_rois_allFix_subj0{sub_num}.csv')
    
    if eytracking_df.empty:
        print(f"No data for subject {sub_num}")
        continue
   
    # Map ROI to category
    roi_to_category = {}
    for category, (rois, _) in roi_categories.items():
        for roi in rois:
            roi_to_category[roi] = category
    eytracking_df['roi_category'] = eytracking_df['ROI'].map(roi_to_category)

    # Create a grouped ROI order: all ROIs grouped by category order
    grouped_roi_order = []
    for category, (rois, _) in roi_categories.items():
        grouped_roi_order.extend(rois)
    # Filter to only those present in the DataFrame
    grouped_roi_order = [roi for roi in grouped_roi_order if roi in eytracking_df['ROI'].values]


    # Use hue and a palette mapping category to color
    category_palette = {cat: color for cat, (_, color) in roi_categories.items()}

    plt.figure(figsize=(20, 8))
    ax = sns.barplot(
        data=eytracking_df, x='ROI', y='r_value',
        hue='roi_category', palette=category_palette,
        estimator=lambda x: np.tanh(np.mean(np.arctanh(x))),
        errorbar='sd',  # Use standard deviation for error bars
        order=grouped_roi_order
    )
    # Ensure gridlines are behind the bars
    ax.set_axisbelow(True)
    # Add vertical gridlines
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    plt.xlabel("ROI")
    plt.ylabel("Mean Correlation (r)")
    plt.title(f"Mean Correlation per ROI - Eye Tracking All Fixations, Subject 0{sub_num}")


    #######################################################################################################################################
    # Define custom ROI label mapping - so that the labels of the plot are coherent with the thesis and literature 
    roi_label_map = {
        'v4': 'hV4',
        'hmt': 'hMT',
        'mtl_faces': 'MTL-faces',
        'mtl_bodies': 'MTL-bodies',
        'mfs_words': 'MFS-words',
        'mtl_words': 'MTL-words',
        'owfa': 'OVWFA',
        # Add more mappings if needed
    }

    # Get current x-tick labels (original ROI names)
    original_roi_order = [label.get_text() for label in ax.get_xticklabels()]

    # Apply custom label mapping for display
    custom_labels = [roi_label_map.get(lbl, lbl.upper()) for lbl in original_roi_order]
    ax.set_xticklabels(custom_labels, rotation=60)

    plt.tight_layout()
    plt.ylim(-0.1,0.37)

    # Move the legend to the top right corner
    if ax.legend_ is not None:
        ax.legend_.set_title("ROI category")
        ax.legend_.set_bbox_to_anchor((1.05, 1))

    plt.xticks(rotation=60)
    plt.ylim(-0.1, 0.37)
    plt.tight_layout()

    # Get the order of ROIs and hues as plotted
    roi_order = [label.get_text() for label in ax.get_xticklabels()]
    hue_order = list(deepgaze_df['roi_category'].dropna().unique())
    n_hue = len(hue_order)
    n_roi = len(roi_order)

    ######################################################################################################################################
    #Load the fdr_corrected p_values per ROI and subject from the subjectwiese df geneated in 03_analysis, while performing the ttest
    # Create a lookup for p-values

    for sub_num_inner_loop in range(1, 9): # Renamed to avoid conflict
        # Define the subject-specific results path dynamically 
        RESULTS_PATH_subj = RESULTS_PATH / f'subj0{sub_num_inner_loop}'
        # Load the DataFrame with FDR-corrected p-values
        # Assuming sal_model is defined elsewhere or you want to use a specific one
        current_sal_model = 'AllFixations' # Placeholder, adjust as needed
        df_singlesub_fdr= pd.read_csv(RESULTS_PATH_subj / f'roi_means_subj0{sub_num_inner_loop}_{current_sal_model}_fdr_corrected.csv', index_col=False)
    
    # Add significance stars using p-values
    for i, bar in enumerate(ax.patches[:len(original_roi_order)]):
        roi = original_roi_order[i]
        match = df_singlesub_fdr[df_singlesub_fdr['ROI'] == roi]
        
        # 
        if match.empty or pd.isnull(match['FDR-corrected p-value'].values[0]):
            continue
        p_value = match['FDR-corrected p-value'].values[0]
        # Only print stars if the correlation (bar height) is positive
        if bar.get_height() > 0:
            if p_value < 0.001:
                significance = '***'
            elif p_value < 0.01:
                significance = '**'
            elif p_value < 0.05:
                significance = '*'
            else:
                significance = ''
            if significance:
                ax.text(
                    bar.get_x() + bar.get_width() / 2,
                    bar.get_height() + 0.01, # Adjusted y-offset
                    significance,
                    ha='center', va='bottom',
                    fontsize=14, color='red'
                )

    plt.savefig(RESULTS_PATH / f'eyetracking_AllFixations_mean_correlation_sig_subj0{sub_num}.png', dpi=300, bbox_inches='tight')
    plt.savefig(RESULTS_PATH / f'eyetracking_AllFixations_mean_correlation_sig_subj0{sub_num}.svg', bbox_inches='tight')
    plt.show()

In [ ]:
for sub_num in range(1, 9):
    # Subjectwise Data Path Storage
    DATA_PATH_subj = DATA_PATH / f'subjectwise_data/subject0{sub_num}'
    # Filter for DeepGaze and the current subject
    deepgaze_df = pd.read_csv(DATA_PATH_subj / f'corr_mat_all_rois_deepgazeIIE_subj0{sub_num}.csv')
    if deepgaze_df.empty:
        print(f"No data for subject {sub_num}")
        continue

    # Map ROI to category
    roi_to_category = {}
    for category, (rois, _) in roi_categories.items():
        for roi in rois:
            roi_to_category[roi] = category
    deepgaze_df['roi_category'] = deepgaze_df['ROI'].map(roi_to_category)

    # Create a grouped ROI order: all ROIs grouped by category order
    grouped_roi_order = []
    for category, (rois, _) in roi_categories.items():
        grouped_roi_order.extend(rois)
    # Filter to only those present in the DataFrame
    grouped_roi_order = [roi for roi in grouped_roi_order if roi in deepgaze_df['ROI'].values]

    # Use hue and a palette mapping category to color
    category_palette = {cat: color for cat, (_, color) in roi_categories.items()}

    plt.figure(figsize=(20, 8))
    ax = sns.barplot(
        data=deepgaze_df, x='ROI', y='r_value',
        hue='roi_category', palette=category_palette,
        estimator=lambda x: np.tanh(np.mean(np.arctanh(x))),
        errorbar='sd',
        order=grouped_roi_order  # Use the sorted ROIs defined earlier
    )
    # Ensure gridlines are behind the bars
    ax.set_axisbelow(True)
    # Add vertical gridlines
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    plt.xlabel("ROI")
    plt.ylabel("Mean Correlation (r)")
    plt.title(f"Mean Correlation per ROI - DeepGaze, Subject 0{sub_num}")

    # Define custom ROI label mapping
    roi_label_map = {
        'v4': 'hV4',
        'hmt': 'hMT',
        'mtl_faces': 'MTL-faces',
        'mtl_bodies': 'MTL-bodies',
        'mfs_words': 'MFS-words',
        'mtl_words': 'MTL-words',
        'owfa': 'OVWFA',
    }

    # Get current x-tick labels (original ROI names)
    original_roi_order = [label.get_text() for label in ax.get_xticklabels()]
    custom_labels = [roi_label_map.get(lbl, lbl.upper()) for lbl in original_roi_order]
    ax.set_xticklabels(custom_labels, rotation=60)

    #plt.tight_layout()
    plt.ylim(-0.1, 0.37)

    # Move the legend to the top right corner
    if ax.legend_ is not None:
        ax.legend_.set_bbox_to_anchor((1.05, 1))

    ######################################################################################################################################
    # Load the fdr_corrected p_values per ROI and subject from the subjectwise df geneated in 03_analysis, while performing the t-test
    # Create a lookup for p-values

    for sub_num_inner_loop in range(1, 9): # Renamed to avoid conflict
        # Define the subject-specific results path dynamically 
        RESULTS_PATH_subj = RESULTS_PATH / f'subj0{sub_num_inner_loop}'
        # Load the DataFrame with FDR-corrected p-values
        # Assuming sal_model is defined elsewhere or you want to use a specific one
        current_sal_model = 'DeepGaze' # Placeholder, adjust as needed
        df_singlesub_fdr= pd.read_csv(RESULTS_PATH_subj / f'roi_means_subj0{sub_num_inner_loop}_{current_sal_model}_fdr_corrected.csv', index_col=False)

    # Save the updated DataFrame with FDR correction results
    
    # Add significance stars using p-values
    for i, bar in enumerate(ax.patches[:len(original_roi_order)]):
        roi = original_roi_order[i]
        match = df_singlesub_fdr[df_singlesub_fdr['ROI'] == roi]
        
        # 
        if match.empty or pd.isnull(match['FDR-corrected p-value'].values[0]):
            continue
        p_value = match['FDR-corrected p-value'].values[0]

        # Only print stars if the correlation (bar height) is positive
        if bar.get_height() > 0:
            if p_value < 0.001:
                significance = '***'
            elif p_value < 0.01:
                significance = '**'
            elif p_value < 0.05:
                significance = '*'
            else:
                significance = ''
            if significance:
                ax.text(
                    bar.get_x() + bar.get_width() / 2,
                    bar.get_height() + 0.01, # Adjusted y-Offset
                    significance,
                    ha='center', va='bottom',
                    fontsize=14, color='red'
                )

    plt.savefig(RESULTS_PATH / f'deepgaze_mean_correlation_sig_subj0{sub_num}.png', dpi=300, bbox_inches='tight')
    plt.savefig(RESULTS_PATH / f'deepgaze_mean_correlation_sig_subj0{sub_num}.svg', bbox_inches='tight')
    plt.show()

## Rank df_mean_roi_stats for the highest and lowest ROI value 

In [ ]:
# Print the rois with th 4 highest mean r values
# Sort the DataFrame by mean r value in descending order
df_mean_roi_stats_ranked = df_mean_roi_stats.sort_values(by='Mean_r', ascending=False)
# Print the top 4 ROIs with the highest mean r values
top_rois = df_mean_roi_stats_ranked.head(4)
print("Top 4 ROIs with highest mean r values:")
print(top_rois)

In [ ]:
# Print ranked 
df_mean_roi_stats_ranked

In [ ]:
all_df

In [ ]:
# Save the aggregated, ranked DataFrame to a CSV file  
# df_mean_roi_stats_ranked.to_csv(RESULTS_PATH / f'df_mean_roi_stat_ranked_all_subjects_{sal_model}_fdr_corrected_ranked.csv', index=False)

## Paired t-tests: Is the correlation of the two saliency models (DeepGaze IIE or GroudTruthEyeTrackingData) significantly different?

Compute the mean and STD deviance of the model difference per ROI. 

In [ ]:
""" # Compute model difference per ROI per subject
model_diff_results = []

for roi in all_df['ROI'].unique():
    dg = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'DeepGaze')][['subject', 'pearsonr']]
    et = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'GroundTruthEyeTracking')][['subject', 'pearsonr']]
    merged = pd.merge(dg, et, on='subject', suffixes=('_dg', '_et'))
    if not merged.empty:
        merged['model_diff'] = merged['pearsonr_dg'] - merged['pearsonr_et']
        for _, row in merged.iterrows():
            model_diff_results.append({
                'ROI': roi,
                'subject': row['subject'],
                'model_diff': row['model_diff']
            })

model_diff_df = pd.DataFrame(model_diff_results)
print(model_diff_df.head())

# To get the mean and std of the difference per ROI:
roi_diff_summary = model_diff_df.groupby('ROI')['model_diff'].agg(['mean', 'std', 'count']).reset_index()
print(roi_diff_summary) """

## Repeated Measures ANOVA to evaluate the performance of all three models 

Since we implemented another model. Let's perform a repeated measures ANOVA - this is appropiate two compare three or more models. 

- If the ANOVA is significant, I can follow up with posthoc- pairwise comparisons in the (with correction) 

First small test for a single roi 

In [ ]:
# Filter to only the models you want to compare (e.g., three models)
models_of_interest = ['GroundTruthEyeTracking', 'AllFixations', 'DeepGaze']
df_anova = all_df[all_df['sal_model'].isin(models_of_interest)].copy()

# Example: repeated measures ANOVA for a single ROI
roi = 'v1'
df_roi = df_anova[df_anova['ROI'] == roi]

# Wide format for ANOVA
df_wide = df_roi.pivot(index='subject', columns='sal_model', values='pearsonr').dropna()

# Melt to long format for statsmodels
df_long = df_wide.reset_index().melt(id_vars='subject', var_name='sal_model', value_name='pearsonr')

# Run repeated measures ANOVA
aovrm = sm.stats.AnovaRM(df_long, 'pearsonr', 'subject', within=['sal_model'])
anova_results = aovrm.fit()
print(anova_results)

Then for all ROIs 

In [ ]:
# Filter to only the models you want to compare (e.g., three models)
models_of_interest = ['GroundTruthEyeTracking', 'AllFixations', 'DeepGaze']
df_anova = all_df[all_df['sal_model'].isin(models_of_interest)].copy()

anova_results_list = []

for roi in sorted_rois:
    df_roi = df_anova[df_anova['ROI'] == roi]
    df_wide = df_roi.pivot(index='subject', columns='sal_model', values='pearsonr').dropna()
    if df_wide.shape[0] < 2:  # Need at least 2 subjects for ANOVA
        continue
    df_long = df_wide.reset_index().melt(id_vars='subject', var_name='sal_model', value_name='pearsonr')
    try:
        aovrm = sm.stats.AnovaRM(df_long, 'pearsonr', 'subject', within=['sal_model'])
        anova = aovrm.fit()
        p_value = anova.anova_table['Pr > F'][0]
        f_value = anova.anova_table['F Value'][0]
        num_df = anova.anova_table['Num DF'][0]
        den_df = anova.anova_table['Den DF'][0]

         # Calculate partial eta squared from the F-value, and the two degrees of freedom associated with an F-test 
         # nominater and denominater degrees of freedom
         # according to Daniel Lakens 
         #  https://sites.google.com/site/lakens2/blog/thefirstruleofnotunderstandingeffectsizesisyoudon%E2%80%99ttalkaboutnotunderstandingeffectsizes
        ms_effect =  f_value * num_df # MS_effect = F Value times (k-1) K = number of saliency models 
        #df_d = 8-1  # df_d = n_subjects - 1
        #partial_eta_squared = ms_effect / (ms_effect + df_d)

        partial_eta_squared = ms_effect / (ms_effect + den_df)
        partial_eta_squared = f_value / (f_value + (den_df / num_df))

        # Correct SEM calculation
        #sem = df_wide.std(axis=0) / np.sqrt(df_wide.shape[0])  # SD / sqrt(n)
        #sem = sem.reset_index()
        #sem.rename(columns={0: 'SEM'}, inplace=True)

        # Calculate SEM and confidence intervals for each saliency model
        ci_results = []
        for sal_model in df_long['sal_model'].unique():
            group_data = df_long[df_long['sal_model'] == sal_model]['pearsonr']
            mean = group_data.mean()
            sem = group_data.std(ddof=1) / np.sqrt(len(group_data))  # Standard error
            df = len(group_data) - 1  # Degrees of freedom
            t_critical = t.ppf(0.975, df)  # 95% confidence level
            lower_ci = mean - t_critical * sem
            upper_ci = mean + t_critical * sem
            ci_results.append({'sal_model': sal_model, 'mean_r': mean, 'lower_ci': lower_ci, 'upper_ci': upper_ci})

        """       # Also report the confidencand were is the sem in the dataframe
        e intervals
        # Calculate the 95% confidence intervals for each model
        ci = df_long.groupby('sal_model')['pearsonr'].agg(lambda x: np.percentile(x, [2.5, 97.5])).reset_index()
        ci[['lower_ci', 'upper_ci']] = pd.DataFrame(ci['pearsonr'].tolist(), index=ci.index)
        ci.drop(columns=['pearsonr'], inplace=True)
        # Merge CI back to the original DataFrame
        df_long = df_long.merge(ci, on='sal_model', how='left') """
 

        anova_results_list.append({
        'ROI': roi, 
        'F': f_value, 
        'p': p_value, 
        'partial_eta_squared': partial_eta_squared,
        'n_subjects': df_wide.shape[0],
        'CI': ci_results  # Store confidence intervals for each saliency model
        })
    except Exception as e:
        print(f"ANOVA failed for ROI {roi}: {e}")

anova_results_df = pd.DataFrame(anova_results_list)

# Apply FDR correction to the ANOVA p-values
if not anova_results_df.empty and 'p' in anova_results_df.columns:
    p_values_anova = anova_results_df['p'].dropna()
    if len(p_values_anova) > 0:
        reject, corrected_p_values_anova = multipletests(p_values_anova, method='fdr_bh')[:2]
        
        # Create a temporary DataFrame for corrected p-values to merge safely
        # Handles cases where dropna() might change indices or length
        temp_corrected_df = pd.DataFrame({
            'p': p_values_anova, 
            'fdr_p_value': corrected_p_values_anova,
            'significant_fdr': reject
        })
        
        # Merge corrected p-values back into the original DataFrame
        # Ensure we are merging on the original 'p' values to align correctly
        # This requires 'p' to be unique for each row, or use index if alignment is guaranteed
        # If 'p' values are not unique, merging on index might be safer if p_values_anova maintains original indexing
        
        # Assuming p_values_anova maintains original index from anova_results_df where 'p' was not NaN:
        original_indices = p_values_anova.index
        temp_corrected_df.index = original_indices

        anova_results_df = anova_results_df.join(temp_corrected_df[['fdr_p_value', 'significant_fdr']])

print(anova_results_df)

# Optionally, save results
anova_results_df.to_csv(RESULTS_PATH / 'anova_results_per_roi.csv', index=False)

In [ ]:
#aovrm = sm.stats.AnovaRM(df_long, 'pearsonr', 'subject', within=['sal_model'])
#anova = aovrm.fit()

In [ ]:
#df_long['sal_model'].nunique() - 1

In [ ]:
""" f = anova.anova_table['F Value'][0]
df_n = anova.anova_table['Num DF'][0]
df_d = anova.anova_table['Den DF'][0]

partial_eta_squared = f * df_n / (f * df_n + df_d)
print(f"Partial eta squared: {partial_eta_squared}") """

In [ ]:
#anova.anova_table

In [ ]:
print(all_df['sal_model'].unique())

## Post-hoc: Paired t-tests (undirected), for each ROI, of the r values for DeepGaze and GroundTruthEyeTracking differ significantly across subjects.

only for the ROIs that were significant 

DG - FirstFix

In [ ]:
paired_ttest_results = []

for roi in all_df['ROI'].unique():
    dg = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'DeepGaze')][['subject', 'pearsonr']]
    et = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'GroundTruthEyeTracking')][['subject', 'pearsonr']]
    merged = pd.merge(dg, et, on='subject', suffixes=('_dg', '_et'))
    if len(merged) > 1:
    # Only consider fdr corercted p-values smaller than 0.05, labeled as significant
        # get the significant_fdr values from the anova_results_df
        significant_fdr = anova_results_df[anova_results_df['ROI'] == roi]['significant_fdr'].values[0]
        if significant_fdr:
            # Perform paired t-test
            t_stat, p_value = ttest_rel(merged['pearsonr_dg'], merged['pearsonr_et'])
            if p_value < 0.05:
                paired_ttest_results.append({
                    'ROI': roi,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'n_subjects': len(merged)
                })

paired_ttest_results_df = pd.DataFrame(paired_ttest_results)
# Apply FDR correction to the paired t-test p-values
multipletests(paired_ttest_results_df['p_value'], method='fdr_bh')
# Create a temporary DataFrame for corrected p-values to merge safely
corrected_p_values = multipletests(paired_ttest_results_df['p_value'], method='fdr_bh')[1]
# Merge corrected p-values back into the original DataFrame
paired_ttest_results_df['fdr_p_value'] = corrected_p_values
# Add a column to indicate significance based on FDR-corrected p-values
paired_ttest_results_df['significant_fdr'] = paired_ttest_results_df['fdr_p_value'] < 0.05
#Caluculate cohens d for the paired t-test results: d= t/squarootn
paired_ttest_results_df['cohens_d'] = paired_ttest_results_df['t_stat'] / (paired_ttest_results_df['n_subjects'] ** 0.5)


print(paired_ttest_results_df)

# Save the paired t-test results
paired_ttest_results_df.to_csv(RESULTS_PATH / 'paired_ttest_results_etFirst_dg.csv', index=False)

AllFix - DG

In [ ]:
paired_ttest_results = []

for roi in all_df['ROI'].unique():
    
    allFix = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'AllFixations')][['subject', 'pearsonr']]
    dg = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'DeepGaze')][['subject', 'pearsonr']]
    merged = pd.merge(allFix, dg, on='subject', suffixes=('_allFix', '_dg'))
    if len(merged) > 1:

    # Only consider fdr corercted p-values smaller than 0.05, labeled as significant
        # get the significant_fdr values from the anova_results_df
        significant_fdr = anova_results_df[anova_results_df['ROI'] == roi]['significant_fdr'].values[0]
        if significant_fdr:
            # Perform undirect pair t-test 
            t_stat, p_value = ttest_rel( merged['pearsonr_allFix', merged['pearsonr_dg'],])
            if p_value < 0.05:
                paired_ttest_results.append({
                    'ROI': roi,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'n_subjects': len(merged)
                })

paired_ttest_results_df = pd.DataFrame(paired_ttest_results)
# Apply FDR correction to the paired t-test p-values
multipletests(paired_ttest_results_df['p_value'], method='fdr_bh')
# Create a temporary DataFrame for corrected p-values to merge safely
corrected_p_values = multipletests(paired_ttest_results_df['p_value'], method='fdr_bh')[1]
# Merge corrected p-values back into the original DataFrame
paired_ttest_results_df['fdr_p_value'] = corrected_p_values
# Add a column to indicate significance based on FDR-corrected p-values
paired_ttest_results_df['significant_fdr'] = paired_ttest_results_df['fdr_p_value'] < 0.05
#Caluculate cohens d for the paired t-test results: d= t/squarootn
paired_ttest_results_df['cohens_d'] = paired_ttest_results_df['t_stat'] / (paired_ttest_results_df['n_subjects'] ** 0.5)

print(paired_ttest_results_df)
# Save the paired t-test results
paired_ttest_results_df.to_csv(RESULTS_PATH / 'paired_ttest_results_allFix_dg.csv', index=False)

AllFix - FirstFix

In [ ]:
paired_ttest_results = []

for roi in all_df['ROI'].unique():
    
    
    allFix = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'AllFixations')][['subject', 'pearsonr']]
    et = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'GroundTruthEyeTracking')][['subject', 'pearsonr']]
    
    merged = pd.merge(allFix, et, on='subject', suffixes=('_allFix', '_et'))
    if len(merged) > 1:
    # Only consider fdr corrected p-values smaller than 0.05, labeled as significant
        # get the significant_fdr values from the anova_results_df
        significant_fdr = anova_results_df[anova_results_df['ROI'] == roi]['significant_fdr'].values[0]
        if significant_fdr:
            # Perform paired t-test
            t_stat, p_value = ttest_rel(merged['pearsonr_et'], merged['pearsonr_allFix'])
            if p_value < 0.05:
                paired_ttest_results.append({
                    'ROI': roi,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'n_subjects': len(merged)
                })

paired_ttest_results_df = pd.DataFrame(paired_ttest_results)
# Apply FDR correction to the paired t-test p-values
multipletests(paired_ttest_results_df['p_value'], method='fdr_bh')
# Create a temporary DataFrame for corrected p-values to merge safely
corrected_p_values = multipletests(paired_ttest_results_df['p_value'], method='fdr_bh')[1]
# Merge corrected p-values back into the original DataFrame
paired_ttest_results_df['fdr_p_value'] = corrected_p_values
# Add a column to indicate significance based on FDR-corrected p-values
paired_ttest_results_df['significant_fdr'] = paired_ttest_results_df['fdr_p_value'] < 0.05
#Caluculate cohens d for the paired t-test results: d= t/squarootn
paired_ttest_results_df['cohens_d'] = paired_ttest_results_df['t_stat'] / (paired_ttest_results_df['n_subjects'] ** 0.5)

print(paired_ttest_results_df)
# Save the paired t-test results
paired_ttest_results_df.to_csv(RESULTS_PATH / 'paired_ttest_results_etFirst_allFix.csv', index=False)

## Model comparison averaged over all ROIs undirected paired t-tests 

AllFix - FirstFix

In [ ]:
# Averaged over all ROIs paired t-test two tailed 

# Ensure both models have the same number of subjects and ROIs = paired observations
merged = pd.merge(
    all_df[all_df['sal_model'] == 'AllFixations'][['subject', 'ROI', 'pearsonr']],
    all_df[all_df['sal_model'] == 'GroundTruthEyeTracking'][['subject', 'ROI', 'pearsonr']],
    on=['subject', 'ROI'],
    suffixes=('_allFix', '_et')
)

print(f"Number of n: {len(merged)}")

# Perform the two-tailed paired t-test
t_stat, p_value = ttest_rel(merged['pearsonr_allFix'], merged['pearsonr_et'])
# Add cohens d for the averaged results 
cohens_d = t_stat / (len(merged) ** 0.5)

# Print the results
print(f"Paired t-test across all ROIs:")
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")

# Check significance
if p_value < 0.05:
    print("The results are significant across all ROIs.")
else:
    print("The results are not significant across all ROIs.")




Same for DG and FirstFix

In [ ]:
# Averaged over all ROIs paired t-test two tailed 

# Ensure both models have the same number of subjects and ROIs
merged = pd.merge(
    all_df[all_df['sal_model'] == 'DeepGaze'][['subject', 'ROI', 'pearsonr']],
    all_df[all_df['sal_model'] == 'GroundTruthEyeTracking'][['subject', 'ROI', 'pearsonr']],
    on=['subject', 'ROI'],
    suffixes=('_dg', '_et')
)

# Perform the two-tailed paired t-test
t_stat, p_value = ttest_rel(merged['pearsonr_dg'], merged['pearsonr_et'])
# Add cohens d for the averaged results 
cohens_d = t_stat / (len(merged) ** 0.5)

# Print the results
print(f"Paired t-test across all ROIs:")
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")

# Check significance
if p_value < 0.05:
    print("The results are significant across all ROIs.")
else:
    print("The results are not significant across all ROIs.")

Same for DG and AllFix

In [ ]:
# Averaged over all ROIs paired t-test two tailed 

# Ensure both models have the same number of subjects and ROIs
merged = pd.merge(
    all_df[all_df['sal_model'] == 'DeepGaze'][['subject', 'ROI', 'pearsonr']],
    all_df[all_df['sal_model'] == 'AllFixations'][['subject', 'ROI', 'pearsonr']],
    on=['subject', 'ROI'],
    suffixes=('_dg', '_allFix')
)

# Perform the two-tailed paired t-test
t_stat, p_value = ttest_rel(merged['pearsonr_dg'], merged['pearsonr_allFix'])
# Add cohens d for the averaged results 
cohens_d = t_stat / (len(merged) ** 0.5)

# Print the results
print(f"Paired t-test across all ROIs:")
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")

# Check significance
if p_value < 0.05:
    print("The results are significant across all ROIs.")
else:
    print("The results are not significant across all ROIs.")

### Now execution of directed t-tests 

Now perform paired (directed) t-tests to test for the direction of the differences in between the prediction of the saliency models. 


In [ ]:
# Perform a directed paired t-test between the two models
paired_ttest_results_directed = []

for roi in all_df['ROI'].unique():

    allFix = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'AllFixations')][['subject', 'pearsonr']]
    et = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'GroundTruthEyeTracking')][['subject', 'pearsonr']]
    
    merged = pd.merge(allFix, et, on='subject', suffixes=('_allFix', '_et'))
    if len(merged) > 1:
    # Only consider fdr corrected p-values smaller than 0.05, labeled as significant
        # get the significant_fdr values from the anova_results_df
        significant_fdr = anova_results_df[anova_results_df['ROI'] == roi]['significant_fdr'].values[0]
        if significant_fdr:
            # Perform directed paired t-test 
            t_stat, p_value = ttest_rel(merged['pearsonr_allFix'], merged['pearsonr_et'], alternative='greater')
            if p_value < 0.05:
                paired_ttest_results_directed.append({
                    'ROI': roi,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'n_subjects': len(merged)
                })

paired_ttest_results_df_directed = pd.DataFrame(paired_ttest_results_directed)
# Apply FDR correction to the paired t-test p-values
multipletests(paired_ttest_results_df_directed['p_value'], method='fdr_bh')
# Create a temporary DataFrame for corrected p-values to merge safely
corrected_p_values = multipletests(paired_ttest_results_df_directed['p_value'], method='fdr_bh')[1]
# Merge corrected p-values back into the original DataFrame
paired_ttest_results_df_directed['fdr_p_value'] = corrected_p_values
# Add a column to indicate significance based on FDR-corrected p-values
paired_ttest_results_df_directed['significant_fdr'] = paired_ttest_results_df_directed['fdr_p_value'] < 0.05
#Caluculate cohens d for the paired t-test results: d= t/squarootn
paired_ttest_results_df_directed['cohens_d'] = paired_ttest_results_df_directed['t_stat'] / (paired_ttest_results_df_directed['n_subjects'] ** 0.5)

print(paired_ttest_results_df_directed)
# Save the paired t-test results
paired_ttest_results_df_directed.to_csv(RESULTS_PATH / 'paired_ttest_results_directed_allFix_firstFix.csv', index=False)

Directed paired t-test for DeepGaze IIE and First-Fixation 

In [ ]:
# Perform a directed paired t-test between the two models
paired_ttest_results_directed = []

for roi in all_df['ROI'].unique():

    dg = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'DeepGaze')][['subject', 'pearsonr']]
    et = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'GroundTruthEyeTracking')][['subject', 'pearsonr']]
    merged = pd.merge(dg, et, on='subject', suffixes=('_dg', '_et'))
    if len(merged) > 1:
    # Only consider fdr corrected p-values smaller than 0.05, labeled as significant
        # get the significant_fdr values from the anova_results_df
        significant_fdr = anova_results_df[anova_results_df['ROI'] == roi]['significant_fdr'].values[0]
        if significant_fdr:
            # Perform directed paired t-test 
            t_stat, p_value = ttest_rel(merged['pearsonr_dg'], merged['pearsonr_et'], alternative='greater')
            if p_value < 0.05:
                paired_ttest_results_directed.append({
                    'ROI': roi,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'n_subjects': len(merged)
                })

paired_ttest_results_df_directed = pd.DataFrame(paired_ttest_results_directed)
# Apply FDR correction to the paired t-test p-values
multipletests(paired_ttest_results_df_directed['p_value'], method='fdr_bh')
# Create a temporary DataFrame for corrected p-values to merge safely
corrected_p_values = multipletests(paired_ttest_results_df_directed['p_value'], method='fdr_bh')[1]
# Merge corrected p-values back into the original DataFrame
paired_ttest_results_df_directed['fdr_p_value'] = corrected_p_values
# Add a column to indicate significance based on FDR-corrected p-values
paired_ttest_results_df_directed['significant_fdr'] = paired_ttest_results_df_directed['fdr_p_value'] < 0.05
#Caluculate cohens d for the paired t-test results: d= t/squarootn
paired_ttest_results_df_directed['cohens_d'] = paired_ttest_results_df_directed['t_stat'] / (paired_ttest_results_df_directed['n_subjects'] ** 0.5)

print(paired_ttest_results_df_directed)
# Save the paired t-test results
paired_ttest_results_df_directed.to_csv(RESULTS_PATH / 'paired_ttest_results_directed_DG_firstFix.csv', index=False)

Directed paired t-test for DG and All Fix

In [ ]:
# Perform a directed paired t-test between the two models
paired_ttest_results_directed = []

for roi in all_df['ROI'].unique():

    dg = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'DeepGaze')][['subject', 'pearsonr']]
    allFix = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == 'AllFixations')][['subject', 'pearsonr']]
    
    merged = pd.merge(dg, allFix, on='subject', suffixes=('_dg','_allFix'))
    if len(merged) > 1:
    # Only consider fdr corrected p-values smaller than 0.05, labeled as significant
        # get the significant_fdr values from the anova_results_df
        significant_fdr = anova_results_df[anova_results_df['ROI'] == roi]['significant_fdr'].values[0]
        if significant_fdr:
            # Perform directed paired t-test 
            t_stat, p_value = ttest_rel(merged['pearsonr_dg'], merged['pearsonr_allFix'], alternative='greater')
            if p_value < 0.05:
                paired_ttest_results_directed.append({
                    'ROI': roi,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'n_subjects': len(merged)
                })

paired_ttest_results_df_directed = pd.DataFrame(paired_ttest_results_directed)
# Apply FDR correction to the paired t-test p-values
multipletests(paired_ttest_results_df_directed['p_value'], method='fdr_bh')
# Create a temporary DataFrame for corrected p-values to merge safely
corrected_p_values = multipletests(paired_ttest_results_df_directed['p_value'], method='fdr_bh')[1]
# Merge corrected p-values back into the original DataFrame
paired_ttest_results_df_directed['fdr_p_value'] = corrected_p_values
# Add a column to indicate significance based on FDR-corrected p-values
paired_ttest_results_df_directed['significant_fdr'] = paired_ttest_results_df_directed['fdr_p_value'] < 0.05
#Caluculate cohens d for the paired t-test results: d= t/squarootn
paired_ttest_results_df_directed['cohens_d'] = paired_ttest_results_df_directed['t_stat'] / (paired_ttest_results_df_directed['n_subjects'] ** 0.5)

print(paired_ttest_results_df_directed)
# Save the paired t-test results
paired_ttest_results_df_directed.to_csv(RESULTS_PATH / 'paired_ttest_results_directed_DG_allFix.csv', index=False)

Directed t-tests averaged over the whole sample and ROIs

In [ ]:
# Averaged over all ROIs paired t-test two tailed 

# Ensure both models have the same number of subjects and ROIs
merged = pd.merge(
    all_df[all_df['sal_model'] == 'AllFixations'][['subject', 'ROI', 'pearsonr']],
    all_df[all_df['sal_model'] == 'GroundTruthEyeTracking'][['subject', 'ROI', 'pearsonr']],
    on=['subject', 'ROI'],
    suffixes=('_allFix', '_et')
)

# Print n
print(f"Number of n: {len(merged)}")

# Perform the two-tailed paired t-test
t_stat, p_value = ttest_rel(merged['pearsonr_allFix'], merged['pearsonr_et'], alternative = 'greater')
# Add cohens d for the averaged results 
cohens_d = t_stat / (len(merged) ** 0.5)

# Print the results
print(f"Paired t-test across all ROIs:")
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")

# Check significance
if p_value < 0.05:
    print("The results are significant across all ROIs.")
else:
    print("The results are not significant across all ROIs.")

Same for DG and allFix 

In [ ]:
# Averaged over all ROIs paired t-test two tailed 

# Ensure both models have the same number of subjects and ROIs
merged = pd.merge(
    all_df[all_df['sal_model'] == 'DeepGaze'][['subject', 'ROI', 'pearsonr']],
    all_df[all_df['sal_model'] == 'AllFixations'][['subject', 'ROI', 'pearsonr']],
    on=['subject', 'ROI'],
    suffixes=('_dg', '_allFix')
)
# Print n
print(f"Number of n: {len(merged)}")

# Perform the two-tailed paired t-test
t_stat, p_value = ttest_rel(merged['pearsonr_dg'], merged['pearsonr_allFix'], alternative='greater')
# Add cohens d for the averaged results 
cohens_d = t_stat / (len(merged) ** 0.5)

# Print the results
print(f"Paired t-test across all ROIs:")
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")

# Check significance
if p_value < 0.05:
    print("The results are significant across all ROIs.")
else:
    print("The results are not significant across all ROIs.")

Same for DG and FirstFix

In [ ]:
# Averaged over all ROIs paired t-test two tailed 

# Ensure both models have the same number of subjects and ROIs
merged = pd.merge(
    all_df[all_df['sal_model'] == 'DeepGaze'][['subject', 'ROI', 'pearsonr']],
    all_df[all_df['sal_model'] == 'GroundTruthEyeTracking'][['subject', 'ROI', 'pearsonr']],
    on=['subject', 'ROI'],
    suffixes=('_dg', '_et')
)
# Print n
print(f"Number of n: {len(merged)}")

# Perform the two-tailed paired t-test
t_stat, p_value = ttest_rel(merged['pearsonr_dg'], merged['pearsonr_et'], alternative='greater')
# Add cohens d for the averaged results 
cohens_d = t_stat / (len(merged) ** 0.5)

# Print the results
print(f"Paired t-test across all ROIs:")
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")

# Check significance
if p_value < 0.05:
    print("The results are significant across all ROIs.")
else:
    print("The results are not significant across all ROIs.")

In [ ]:
all_df

In [ ]:
df_mean_roi_stats

## Plotting of model difference (Mean r across all ROIs)

In [ ]:
# Define the new saliency model names and their desired order
saliency_model_rename = {
    'GroundTruthEyeTracking': 'Eye Tracking (First Fixation)',
    'AllFixations': 'Eye Tracking (All Fixations)',
    'DeepGaze': 'DeepGaze IIE'
}
desired_order = ['Eye Tracking (First Fixation)', 'Eye Tracking (All Fixations)', 'DeepGaze IIE']

# Rename the saliency models in the DataFrame
all_df['sal_model'] = all_df['sal_model'].map(saliency_model_rename)

sns.set_theme(style="whitegrid")
g = sns.catplot(
    data=all_df, x='ROI', y='pearsonr', hue='sal_model',
    kind='bar', height=6, aspect=2, palette='Set2', hue_order=desired_order,
    estimator=lambda x: np.tanh(np.mean(np.arctanh(x)))
)

g.despine(left=True)
g.set_axis_labels("ROI", "Mean Correlation (r)")
# Access the legend and set its title
if g.legend is not None:
    g.legend.set_title("Saliency Model")

#######################################################################################################################################
# Define custom ROI label mapping - so that the labels of the plot are coherent with the thesis and literature 
roi_label_map = {
    'v4': 'hV4',
    'hmt': 'hMT',
    'mtl_faces': 'MTL-faces',
    'mtl_bodies': 'MTL-bodies',
    'mfs_words': 'MFS-words',
    'mtl_words': 'MTL-words',
    'owfa': 'OVWFA',
}

# Get current x-tick labels (original ROI names)
ax = g.ax if hasattr(g, 'ax') else g.axes.flat[0]  # Access the correct axis
original_roi_order = [label.get_text() for label in ax.get_xticklabels()]

# Apply custom label mapping for display
custom_labels = [roi_label_map.get(lbl, lbl.upper()) for lbl in original_roi_order]
ax.set_xticklabels(custom_labels, rotation=60)

plt.tight_layout()
plt.ylim(-0.1,0.18)

ax = g.ax

# Move the legend to the top right corner
if g.legend is not None:
    g.legend.set_bbox_to_anchor((1.05, 1))

# Use original ROI names for lookup, but display custom labels
roi_order = original_roi_order
saliency_order = [t.get_text() for t in g.legend.texts]
n_roi = len(roi_order)
n_sal = len(saliency_order)
#######################################################################################################################################
# Look for the FDR corrected p-values in the all_df dataframe
for i, bar in enumerate(ax.patches):
    roi_idx = i % n_roi
    sal_idx = i // n_roi
    if roi_idx >= n_roi or sal_idx >= n_sal:
        continue
    roi = roi_order[roi_idx]  # This is the original ROI name
    sal_model = saliency_order[sal_idx]
    # Try to get the p-value, skip if missing
    match = all_df[(all_df['ROI'] == roi) & (all_df['sal_model'] == sal_model)]
    if match.empty or pd.isnull(match['fdr_p_value'].values[0]):
        continue
    p_value = match['fdr_p_value'].values[0]
    # Determine significance stars
    if p_value < 0.001:
        significance = '***'
    elif p_value < 0.01:
        significance = '**'
    elif p_value < 0.05:
        significance = '*'
    else:
        significance = ''

    # Add the stars above the bar
    if significance:
        g.ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05,
                significance, ha='center', va='bottom', fontsize=12, color='red')
        
# Save the plot to a file
plt.savefig(RESULTS_PATH / 'combined_mean_correlation_sig_all_subjects_fdr_corrected.png', dpi=300, bbox_inches='tight')
# Save as SVG
plt.savefig(RESULTS_PATH / 'combined_mean_correlation_sig_all_subjects_fdr_corrected.svg', bbox_inches='tight')

plt.show()